# Introduction to multivariable Green functions

In this notebook, we learn how to create and manipulate multivariable Green functions.
As an example, we consider the Green's function on a square lattice with nearest-neighbour hopping $t$,

\begin{equation}
G(\mathbf{k},i\omega_n)=\frac{1}{i\omega_n  + \mu - \epsilon(\mathbf{k})}
\end{equation}

with dispersion $\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y})$. Here $\mathbf{k}$ is a vector in the Brillouin zone (in units where the lattice spacing is unity $a=1$), $\mu$ is the chemical potential and $i\omega_n$ is a Matsubara frequency.

## Imports and parameters

Below we import modules that will be useful in the following. We also set the
parameters of the problem.

In [ ]:
# Relevant Imports 
from triqs.lattice import BravaisLattice, BrillouinZone
from triqs.gf import Gf, MeshProduct, MeshBrZone, MeshImFreq
import numpy as np
from math import cos, pi

In [ ]:
# Physical parameters
beta = 2     # Inverse temperature
t = 1.0      # Hopping (unit of energy)
mu = 0       # Chemical potential

## Constructing and Initializing a Lattice Green function

We first define a simple Bravais lattice (`BravaisLattice`) in 2 dimensions with basis vectors $\hat{e}_x = (1, 0, 0)$ and ${\hat e}_y=(0, 1, 0)$. Given this bravais lattice we construct the reciprocal (momentum) space Brillouin zone (`BrillouinZone`), on which we can then construct a momentum mesh (`MeshBrZone`).

In [ ]:
BL = BravaisLattice([(1,0,0), (0,1,0)]) # Two unit vectors in R3
BZ = BrillouinZone(BL) 

# n_k denotes the number of k-points for each dimension
n_k = 128
k_mesh = MeshBrZone(bz=BZ, n_k=n_k)

The Lattice Green's function is defined on a mesh that is the cartesian product of this momentum mesh and a Matsubara mesh.

$$
G: (\mathbf{k}, i\omega_n) \rightarrow {\mathcal{C}}
$$

To construct this mesh we use the `MeshProduct` provided by TRIQS:

In [ ]:
iw_mesh = MeshImFreq(beta=beta, S='Fermion', n_iw=128)
k_iw_mesh = MeshProduct(k_mesh, iw_mesh)

# Recall that for an empty target_shape G0 has values that are scalars instead of matrices.
G = Gf(mesh=k_iw_mesh, target_shape=[])

To fill the Green's function we construct a function for the dispersion $\epsilon(\mathbf{k})$ and set each element of $G$ by looping over the momentum and frequency meshes.

In [ ]:
#%%timeit
def eps(k):
    return -2*t * (cos(k[0]) + cos(k[1]))

# Loop initialization. Slow..
for k, iw in G.mesh:
    G[k, iw] = 1/(iw + mu - eps(k))

## Numpy Broadcasting

Instead of writing a loop we can use the [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) features of the numpy package to assign directly into the data-array of the Green function object. This approach is a lot faster than writing a loop

In [ ]:
iw_arr = np.array(list(iw_mesh.values()))
k_arr  = np.array(list(k_mesh.values()))
np_eps = np.vectorize(eps, signature='(d)->()')

In [ ]:
#%%timeit
# Vectorized function evaluation
eps_arr = np_eps(k_arr)

# Numpy Broadcasting
G.data[:] = 1.0 / (iw_arr[None,::] + mu - eps_arr[::,None])

## Evaluate the Green function

The Green function object $G(k,i\omega_n)$ can be evaluated like an ordinary Python function
at a given reciprocal vector and Matsubara frequency:

- The reciprocal vector $k$ is a tuple/list/numpy.array of double 
- The Matsubara frequency is an integer $n$, the $n$ in $i\omega_n$

The result will be a linear interpolation on the Brillouin zone 
  with the points on the grid of $G$ around $k$.

Therefore, one can use $g_0$ as any python function of $k$ and $i\omega_n$, 
and forget its precise representation in memory (what is the grid, etc...).
We will use that in the plot functions below.

Example:
Let's evaluate the above Green's function at $\mathbf{k} = (\pi,\pi,0)$ and $i\omega_2$. As $\epsilon((\pi,\pi,0)) = 2t = 2$ and $i\omega_2 = i\frac{(2*2 + 1)\pi}{\beta}$, we check:

In [ ]:
G_eval  = G((pi,pi,0), 2)
G_exact = 1.0/(1j * (2*2+1)*pi/beta - 4)
print(G_eval - G_exact) # Check

## Partial evaluation

Given a function $G(k,i\omega_n)$ it is possible to obtain the function $i\omega_n \rightarrow G(k_0, i\omega_n)$ for a fixed $k_0$:

In [ ]:
k0 = (0.02,0.01,0)      # a k-point as a tuple of 3 floats
Giw = G(k0, all)        # We use the "built-in" function all here as equivalent of :, 
                        # which Python does not allow in ()
    
# Giw is a Green function of the Matsubara frequency only
# It is calculated by k-interpolation of G
print(Giw)

# Giw uses the original Matsubara mesh
assert Giw.mesh == G.mesh[1]

Here `Giw` is obtained through linearly of `G` for the point $k_0$ on the original Brillouin zone grid.

It is simply a Matsubara Green function, which means that you can use all the common methods, such as `density()` or Fourier transforms:

In [ ]:
# This is the density n_k at k=(0.02, 0.01)
print("n_k =",  Giw.density().real)

## Defining a Tight-Binding Hamiltonian

In practice we often know the tight-binding Hamiltonian on our Bravais lattice rather than the analytic dispersion relation.
TRIQS provides the `TightBinding` class for this case:

In [ ]:
from triqs.lattice import TightBinding
?TightBinding

In [ ]:
# Define mapping between displacement vectors and hopping amplitudes
# Matrix structure of the amplitudes is w.r.t. atoms in the unit cell (here only one).
hop= {  (1,0)  :  [[ -t]],       
        (-1,0) :  [[ -t]],     
        (0,1)  :  [[ -t]],
        (0,-1) :  [[ -t]]
     }
TB = TightBinding(bl=BL, hoppings=hop)

# Green function on the k_mesh holding the dispersion values
eps_k = TB.dispersion(k_mesh)[0]

# Initialize the lattice Green function using Numpy Broadcasting
Gtb = G.copy()
Gtb.data[:] = 1.0 / (iw_arr[None,::] + mu - eps_k.data[::,None])

# Check Equality
assert np.linalg.norm((G - Gtb).data) < 1e-12

Let us finally plot the dispersion relation we have obtained

In [ ]:
# Prepare the data
k_grid = k_arr.reshape(n_k,n_k,3)
X = k_grid[...,0]/pi
Y = k_grid[...,1]/pi
Z = eps_k.data.reshape(n_k,n_k)

# Plot the dispersion
from matplotlib import pyplot as plt
%matplotlib inline

fig  = plt.figure(dpi=110)
ax   = plt.axes(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap='coolwarm')
fig.colorbar(surf, shrink=0.5, aspect=10)

ax.set_xlabel(r"$k_x/\pi$")
ax.set_ylabel(r"$k_y/\pi$")
ax.set_title(r"$\epsilon(\mathbf{k})$")